In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import requests
import time


In [ ]:

gwas_file = "../data/processed/AFR/synthetic_v1_GWAS_logistic.assoc.logistic"
gwas_data = pd.read_csv(gwas_file, delim_whitespace=True)

gwas_data = gwas_data.dropna(subset=["P"])
num_tests = gwas_data.shape[0]
bonferroni_threshold = 0.05 / num_tests

gwas_data["-log10(P)"] = -np.log10(gwas_data["P"])

gwas_data["chromosome_position"] = 0
current_position = 0
positions = []

for chr_id in sorted(gwas_data["CHR"].unique()):
    chr_data = gwas_data[gwas_data["CHR"] == chr_id]
    gwas_data.loc[gwas_data["CHR"] == chr_id, "chromosome_position"] = chr_data["BP"] + current_position
    positions.append(current_position + chr_data["BP"].median())
    current_position += chr_data["BP"].max()

plt.figure(figsize=(12, 6))

colors = ["#1f77b4", "#ff7f0e"]  
chromosomes = sorted(gwas_data["CHR"].unique())
for i, chr_id in enumerate(chromosomes):
    chr_data = gwas_data[gwas_data["CHR"] == chr_id]
    plt.scatter(
        chr_data["chromosome_position"],
        chr_data["-log10(P)"],
        color=colors[i % 2],
        s=10
    )

plt.axhline(y=-np.log10(bonferroni_threshold), color="red", linestyle="--", label="Genome-wide significance")
plt.xlabel("Chromosome")
plt.ylabel("-log10(P)")
plt.title("GWAS - HAPNEST - Ancestry: AFR")
plt.xticks(positions, chromosomes)
plt.legend(loc="upper right", fontsize=8)
plt.tight_layout()

plt.savefig("../data/results/manhattan_plot_AFR.png", dpi=300)
plt.show()

In [ ]:
significant_snps = gwas_data[gwas_data["P"] < bonferroni_threshold]
significant_snps.sort_values("P").head(10)

In [ ]:
all_maps = []

for chrom in range(1, 23):
    variant_map = pd.read_csv(f"../data/raw/rsids/rsid_variant_map_list_chr{chrom}.txt", delim_whitespace=True)
    all_maps.append(variant_map)

variant_map_df = pd.concat(all_maps)
variant_map_df = variant_map_df.rename(columns={"id_hg38": "SNP", "rsid": "rsID"})

merged_df = pd.merge(gwas_data, variant_map_df, on="SNP", how="left")

significant = merged_df[merged_df["P"] < bonferroni_threshold]
significant.to_csv("../data/results/AFR_significantSNPs.csv", index=False)

significant.sort_values("P").head(20)


In [ ]:


def get_variant_info(rsid):
    server = "https://rest.ensembl.org"
    ext = f"/variation/human/{rsid}?"
    headers = {"Content-Type": "application/json"}

    r = requests.get(server + ext, headers=headers)

    if not r.ok:
        return None

    data = r.json()
    return {
        "rsID": rsid,
        "most_severe_consequence": data.get("most_severe_consequence", None),
        "gene": ", ".join([x.get("gene_symbol", "") for x in data.get("mappings", []) if x.get("gene_symbol")]),
        "phenotypes": ", ".join(data.get("phenotypes", [])) if "phenotypes" in data else None
    }

annotated_variants = []
rsids = significant["rsID"].dropna().unique()

print(f"🔍 Annotating {len(rsids)} rsIDs...")

for i, rsid in enumerate(rsids[:50]):  
    info = get_variant_info(rsid)
    if info:
        annotated_variants.append(info)
    else:
        annotated_variants.append({
            "rsID": rsid,
            "most_severe_consequence": None,
            "gene": None,
            "phenotypes": None
        })
    time.sleep(0.2)  

annotations_df = pd.DataFrame(annotated_variants)
variant_annotated = pd.merge(variant_map_df, annotations_df, on="rsID", how="left")
#variant_annotated.to_csv("results/variant_map_with_annotations.csv", index=False)
variant_annotated.head(10)

In [ ]:
variant_annotated_2 = variant_annotated.dropna(subset=["most_severe_consequence", "gene", "phenotypes"])
print(variant_annotated_2)

## Meta Analysis Results

In [ ]:
gwas_file = "../data/meta_results.meta"
gwas_data = pd.read_csv(gwas_file, delim_whitespace=True)

gwas_data = gwas_data.dropna(subset=["P"])
num_tests = gwas_data.shape[0]
bonferroni_threshold = 0.05 / num_tests

gwas_data["-log10(P)"] = -np.log10(gwas_data["P"])

gwas_data["chromosome_position"] = 0
current_position = 0
positions = []

for chr_id in sorted(gwas_data["CHR"].unique()):
    chr_data = gwas_data[gwas_data["CHR"] == chr_id]
    gwas_data.loc[gwas_data["CHR"] == chr_id, "chromosome_position"] = chr_data["BP"] + current_position
    positions.append(current_position + chr_data["BP"].median())
    current_position += chr_data["BP"].max()

plt.figure(figsize=(12, 6))

colors = ["#1f77b4", "#ff7f0e"]  
chromosomes = sorted(gwas_data["CHR"].unique())
for i, chr_id in enumerate(chromosomes):
    chr_data = gwas_data[gwas_data["CHR"] == chr_id]
    plt.scatter(
        chr_data["chromosome_position"],
        chr_data["-log10(P)"],
        color=colors[i % 2],
        s=10
    )

plt.axhline(y=-np.log10(bonferroni_threshold), color="red", linestyle="--", label="Genome-wide significance")
plt.xlabel("Chromosome")
plt.ylabel("-log10(P)")
plt.title("Meta Analysis - HAPNEST - Ancestry: EUR & AFR")
plt.xticks(positions, chromosomes)  
plt.legend(loc="upper right", fontsize=8)
plt.tight_layout()

plt.savefig("../data/results/manhattan_plot_meta.png", dpi=300)
plt.show()